In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 800)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

# fp_df_eval_results = 'df_eval_results'
fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.

###################################
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean, s_train=1, s_val=0, s_test=0)
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
###################################

len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 1050, len_df_val: 300, len_df_test: 150 


In [3]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt']
col_add1 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add2 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add3 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add4 = ['SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI']
col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4
print(f'col_add_total: {col_add_total}')

col_add_total: ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI']


In [4]:
# # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results)
# df

In [5]:
# if store_results:  # record results to df
# my_cols = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_CAGR/UI']
# df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
df_eval_sym_freq_results = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)

In [6]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 1

# for training, the number of days to lookback from iloc max-lookback end_train
# days_lookbacks = [15, 30, 60, 120]
# days_lookbacks = [30, 60, 120]
# days_lookbacks = [60, 120]
# days_lookbacks = [120]
# days_lookbacks = [60]
# days_lookbacks = [30]
days_lookbacks = [15]
# days_lookbacks = [15, 30]
# days_lookbacks = [15, 30, 60]
# days_lookbacks = [15, 29]
# days_lookbacks = [15, 31]
# days_lookbacks = [14, 30]
# days_lookbacks = [16, 32]
# days_lookbacks = [14, 28]

days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
# days_eval = 10
days_eval = 5
# days_eval = 4
# days_eval = 3
# days_eval = 2  


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
# syms_start = 1  #  start index of n_top_syms for evaluation

# syms_end = n_top_syms  #  end index of n_top_syms for evaluation
# syms_end = 1  #  end index of n_top_syms for evaluation
# syms_end = 2  #  end index of n_top_syms for evaluation
# syms_end = 3  #  end index of n_top_syms for evaluation
# syms_end = 4  #  end index of n_top_syms for evaluation
# syms_end = 5  #  end index of n_top_syms for evaluation
# syms_end = 6  #  end index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

# freq_cnt = 6  # frequency of symbol in top_set_syms_n_freq 

create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [7]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(232, 247, 252)]
sets_lookback_slices:
[[(232, 247, 252)]]


Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [8]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    if verbose:
      print(f'days lookback: {lookback}')
      print(f'lb_slices:     {lb_slices}')
      print(f'lb_slice:      {lb_slice}')
      print(f'days eval:     {d_eval}')    
      print(f'start_train:   {start_train}')
      print(f'end_train:     {end_train}')
      # print(f'start_eval:    {start_eval}')
      # print(f'end_eval:      {end_eval}')`

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback: 15
lb_slices:     [(232, 247, 252)]
lb_slice:      (232, 247, 252)
days eval:     5
start_train:   232
end_train:     247
perf_ranks: {'period-15': {'r_CAGR/UI': array(['COOP', 'TWLO', 'FATE', 'QURE', 'VIRT', 'BL', 'CNK', 'DDS', 'VECO',
       'PAR', 'VNDA', 'WIX', 'ZBRA', 'WOLF', 'CHGG', 'ITCI', 'ASC',
       'ENSG', 'FIVN', 'RNG'], dtype=object), 'r_CAGR/retnStd': array(['COOP', 'FATE', 'TWLO', 'VIRT', 'QURE', 'BL', 'DDS', 'VECO', 'WIX',
       'PAR', 'ITCI', 'RNG', 'VNDA', 'CNK', 'WOLF', 'ENSG', 'ASC', 'CNX',
       'X', 'AXGN'], dtype=object), 'r_retnStd/UI': array(['CNK', 'QURE', 'COOP', 'DDS', 'TWLO', 'ZBRA', 'BL', 'CHGG', 'FIVN',
       'VIRT', 'NYT', 'LITE', 'FATE', 'VNDA', 'BR', 'PAR', 'JLL', 'HCKT',
       'JBL', 'VECO'], dtype=object)}}
most_common_syms: [('COOP', 3), ('TWLO', 3), ('FATE', 3), ('QURE', 3), ('VIRT', 3), ('BL', 3), ('CNK', 3), ('DDS', 3), ('VECO', 3), ('PAR', 3), ('VNDA', 3), ('WIX', 2), ('ZBRA', 2), ('WOLF', 2), ('CHGG', 2), ('ITCI', 2), ('ASC'

In [9]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    # if verbose:
    #     print(f'sym_freq_cnt_6: {sym_freq_cnt_6}')
    #     print(f'sym_freq_cnt_5: {sym_freq_cnt_5}')
    #     print(f'sym_freq_cnt_4: {sym_freq_cnt_4}')
    #     print(f'sym_freq_cnt_3: {sym_freq_cnt_3}')
    #     print(f'sym_freq_cnt_2: {sym_freq_cnt_2}')

    # return sym_freq_cnt_6, sym_freq_cnt_5, sym_freq_cnt_4, sym_freq_cnt_3, sym_freq_cnt_2
    return l_sym_freq_cnt    

In [10]:
# for top_set_syms_n_freq in grp_top_set_syms_n_freq:
#   l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
#   print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
#   print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
#   print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
#   print(f'sym_freq_3: {l_sym_freq_cnt[3]}')
#   print(f'sym_freq_2: {l_sym_freq_cnt[4]}\n')  

In [11]:
# from yf_utils import _7_perf_eval
from yf_utils import _7_perf_eval_v1

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):

  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}\n')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]
  # _, SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI = _7_perf_eval(df_SPY)
  # print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI: {SPY_retnStd_d_UI[0]:>13,.3f}, {SPY_CAGR_d_retnStd[0]:>13,.3f}, {SPY_CAGR_d_UI[0]:>13,.3f}')
  _df_perf_SPY, SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI, SPY_CAGR = _7_perf_eval_v1(df_SPY)
  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>13,.3f}, {SPY_CAGR_d_retnStd[0]:>13,.3f}, {SPY_CAGR_d_UI[0]:>13,.3f}, {SPY_CAGR[0]:>13,.3f}')





  # drop last list in l_sym_freq_cnt from zip
  zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        # print(f'start_eval: {start_eval}')
        # print(f'end_eval:   {end_eval}')  
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')


      # _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_eval)
      _df_perf, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI, grp_CAGR = _7_perf_eval_v1(df_eval)

      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>13,.3f}, {grp_retnStd_d_UI[1]  :>13,.3f}, {grp_retnStd_d_UI[2]  :>13,.3f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>13,.3f}, {grp_CAGR_d_UI[1]     :>13,.3f}, {grp_CAGR_d_UI[2]     :>13,.3f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>13,.3f}, {grp_CAGR[1]          :>13,.3f}, {grp_CAGR[2]          :>13,.3f}')


      if store_results:  # record results to df
        # row_add = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, grp_CAGR_d_UI[0], grp_CAGR_d_UI[1], grp_CAGR_d_UI[2], SPY_CAGR_d_UI[0]]
        # df_eval_results.loc[len(df_eval_results)] = row_add
        # row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end]
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, sym_freq_cnt]        
        row_add1      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add2      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add3      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add4      = [SPY_retnStd_d_UI[0],   SPY_CAGR_d_retnStd[0], SPY_CAGR_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4
        print(f'row_add_total: {row_add_total}')
        df_eval_sym_freq_results.loc[len(df_eval_sym_freq_results)] = row_add_total
        # print(f'appended row_add to df_eval_results:\n{row_add}\n')
        print(f'appended row_add to df_eval_sym_freq_results:\n{row_add_total}\n')
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 1 max_lookback_slice
max_lookback_slice: (232, 247, 252)
max lookback dates: 2018-02-06, 2018-02-28, 2018-03-07
df_eval dates (inclusive): 2018-02-28 - 2018-03-06
top_set_syms_n_freq: [('BL', 3), ('CNK', 3), ('COOP', 3), ('DDS', 3), ('FATE', 3), ('PAR', 3), ('QURE', 3), ('TWLO', 3), ('VECO', 3), ('VIRT', 3), ('VNDA', 3), ('ASC', 2), ('CHGG', 2), ('ENSG', 2), ('FIVN', 2), ('ITCI', 2), ('RNG', 2), ('WIX', 2), ('WOLF', 2), ('ZBRA', 2)]

sym_freq_6: []
sym_freq_5: []
sym_freq_4: []
sym_freq_3: ['BL', 'CNK', 'COOP', 'DDS', 'FATE', 'PAR', 'QURE', 'TWLO', 'VECO', 'VIRT', 'VNDA']


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:         1.244,        34.120,        42.440,         0.329
start_eval: 247,  date: 2018-02-28
end_eval:   252,  date: 2018-03-07,  df_eval last date: 2018-03-06
frequency count of symbol(s): 3

df_eval_n_SPY:
                   BL        CNK   COOP        DDS   FATE    PAR       QURE       TWLO       VECO       VIRT       VNDA         SPY


c:\Users\ping\MyDrive\py_files\python\py379\yf_utils.py:652: RuntimeWarning: divide by zero encountered in double_scalars
  grp_CAGR_d_UI = [_mean, _std, _mean / _std]
c:\Users\ping\MyDrive\py_files\python\py379\yf_utils.py:656: RuntimeWarning: divide by zero encountered in double_scalars
  grp_CAGR_d_retnStd = [_mean, _std, _mean / _std]
c:\Users\ping\MyDrive\py_files\python\py379\yf_utils.py:660: RuntimeWarning: divide by zero encountered in double_scalars
  grp_retnStd_d_UI = [_mean, _std, _mean / _std]
c:\Users\ping\MyDrive\py_files\python\py379\yf_utils.py:665: RuntimeWarning: divide by zero encountered in double_scalars
  grp_CAGR = [grp_MeanCAGR, grp_StdCAGR, grp_MeanCAGR/grp_StdCAGR]


In [12]:
_df_perf

,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,BL,2018-02-28,2018-03-06,0.015873,0.793783,1.020045e-02,1.629683,47.750692,7.781847e+01
1,CNK,2018-02-28,2018-03-06,0.015873,-0.603277,2.138189e-02,1.055135,-26.740083,-2.821439e+01
2,COOP,2018-02-28,2018-03-06,0.015873,109.352449,6.933538e-03,2.970906,5308.657949,1.577152e+04
3,DDS,2018-02-28,2018-03-06,0.015873,57.460786,1.921256e-02,2.103245,1421.990220,2.990793e+03
4,FATE,2018-02-28,2018-03-06,0.015873,11414.682138,2.305894e-02,3.459639,143084.849811,4.950219e+05
5,PAR,2018-02-28,2018-03-06,0.015873,-0.987362,5.277487e-02,0.825052,-22.676075,-1.870893e+01
6,QURE,2018-02-28,2018-03-06,0.015873,0.763780,1.804990e-02,2.120335,19.956706,4.231491e+01
7,TWLO,2018-02-28,2018-03-06,0.015873,1392.737941,4.472136e-07,53258.463741,58474.399988,3.114257e+09
8,VECO,2018-02-28,2018-03-06,0.015873,11.819351,4.472136e-07,17323.721069,1525.588639,2.642887e+07
9,VIRT,2018-02-28,2018-03-06,0.015873,80.703784,5.309792e-03,5.649738,2690.221904,1.519905e+04


In [13]:
_df_perf_SPY

,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,SPY,2018-02-28,2018-03-06,0.015873,0.329254,0.007758,1.243848,34.120195,42.440345


In [14]:
df = df_eval_n_SPY.copy()
df

,BL,CNK,COOP,DDS,FATE,PAR,QURE,TWLO,VECO,VIRT,VNDA,SPY
Date,,,,,,,,,,,,
2018-02-28,44.000000,39.063946,15.48,71.991333,11.26,11.04,25.420000,34.160000,18.150000,24.516943,18.850000,255.162445
2018-03-01,43.810001,37.531124,15.24,72.573952,11.37,10.92,24.959999,34.959999,18.150000,24.269299,18.700001,251.452209
2018-03-02,45.049999,38.403088,15.72,77.658829,13.27,11.16,26.590000,37.330002,18.250000,24.847137,19.000000,252.748444
2018-03-05,44.270000,38.448978,16.08,74.436676,13.77,10.17,26.610001,38.230000,18.549999,26.456839,19.299999,255.669662
2018-03-06,44.410000,38.494873,16.68,76.793716,13.06,10.30,25.650000,38.320000,18.900000,26.291742,19.299999,256.317810


In [15]:
# pickle_dump(df, path_data_dump, 'df_eval_n_SPY_test_data')
# df = pickle_load(path_data_dump, 'df_eval_n_SPY_test_data')
# df

In [16]:
# from myUtils import symb_perf_stats_vectorized_v8
# (
#   symbols,
#   period_yr,
#   retn,
#   DD,
#   UI,
#   MDD,
#   retnMean,
#   retnStd,
#   retnStd_div_UI,
#   CAGR,
#   CAGR_div_retnStd,
#   CAGR_div_UI,
#   grp_MeanCAGR,
#   grp_StdCAGR,

#   # arr_drawdown,
      
# ) = symb_perf_stats_vectorized_v8(df)

In [17]:
# _cols = np.all(dd==0, axis=0)  # True if column is all 0
# _cols

In [18]:
# print(f'symbols: {symbols}')
# print(f'period_yr: {period_yr}')
# print(f'returns:\n{retn}')
# print(f'drawdown:\n{DD}')
# print(f'UI:\n{UI}')
# print(f'max_drawdown:\n{MDD}')
# print(f'returns_mean:\n{retnMean}')
# print(f'returns_std:\n{retnStd}')
# print(f'returns_std_div_UI:\n{retnStd_div_UI}')
# print(f'CAGR:\n{CAGR}')
# print(f'CAGR_div_returns_std:\n{CAGR_div_retnStd}')
# print(f'CAGR_div_UI:\n{CAGR_div_UI}')
# print(f'grp_mean_CAGR: {grp_MeanCAGR:>12.6f}')
# print(f'grp_std_CAGR:  {grp_StdCAGR:>12.6f}')

In [19]:
df_eval_sym_freq_results.columns

Index(['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI'], dtype='object')

In [20]:
row_add_total

[1,
 '[15]',
 5,
 20,
 3,
 6418.62759319925,
 15618.720598073325,
 0.41095732220160414,
 19350.44432630976,
 42440.94834617264,
 0.45593807585251095,
 285565054.6616161,
 894542655.6874447,
 0.31923022658116057,
 1.2438482787310243,
 34.12019460677955,
 42.440345331610324]

In [21]:
df_eval_sym_freq_results

,n_samples,days_lookbacks,days_eval,n_top_syms,sym_freq_cnt,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,SPY_retnStd/UI,SPY_CAGR/retnStd,SPY_CAGR/UI
0,1,[15],5,20,3,1.001363,0.827045,1.210773,404.943626,1.197256e+03,0.338226,1.367754e+03,3.703679e+03,0.369296,1.427039,52.370143,7.473426e+01
1,1,[15],5,20,3,1.515661,0.937109,1.617380,187.471156,4.295033e+02,0.436484,6.584184e+02,1.169536e+03,0.562974,0.664086,-81.356794,-5.402790e+01
2,1,[15],5,20,3,1.354337,0.569836,2.376714,143.128411,1.777160e+02,0.805377,2.908944e+02,3.568226e+02,0.815235,0.713795,-61.222413,-4.370028e+01
3,1,[15],5,20,3,1.487676,1.091290,1.363227,99.119417,1.990707e+02,0.497911,3.291539e+02,4.333328e+02,0.759587,1.027637,-42.641951,-4.382043e+01
4,1,[15],5,20,3,5.585796,7.459553,0.748811,440.582588,6.151076e+02,0.716269,6.447628e+03,1.363845e+04,0.472754,3.702599,677.590790,2.508847e+03
5,1,[15],5,20,3,0.941567,0.133829,7.035578,-58.699896,1.861487e+01,-3.153387,-5.470091e+01,1.650757e+01,-3.313687,0.867524,-51.585626,-4.475178e+01
6,1,[15],5,20,3,2898.979615,7664.030328,0.378258,485066.773129,1.252713e+06,0.387213,2.391041e+08,6.207190e+08,0.385205,8775.490862,844.953978,7.414886e+06
7,1,[15],5,20,3,17809.298271,23457.031539,0.759231,666661.788115,1.480543e+06,0.450282,8.406682e+09,1.835576e+10,0.457986,0.685706,-53.830785,-3.691210e+01
8,1,[15],5,20,3,6418.627593,15618.720598,0.410957,19350.444326,4.244095e+04,0.455938,2.855651e+08,8.945427e+08,0.319230,1.243848,34.120195,4.244035e+01


In [22]:
# # Creating Empty DataFrame and Storing it in variable df
# df_eval_results = pd.DataFrame(columns=col_add_total)
pickle_dump(df_eval_sym_freq_results, path_data_dump, fp_df_eval_sym_freq_results)
df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
df

,n_samples,days_lookbacks,days_eval,n_top_syms,sym_freq_cnt,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,SPY_retnStd/UI,SPY_CAGR/retnStd,SPY_CAGR/UI
0,1,[15],5,20,3,1.001363,0.827045,1.210773,404.943626,1.197256e+03,0.338226,1.367754e+03,3.703679e+03,0.369296,1.427039,52.370143,7.473426e+01
1,1,[15],5,20,3,1.515661,0.937109,1.617380,187.471156,4.295033e+02,0.436484,6.584184e+02,1.169536e+03,0.562974,0.664086,-81.356794,-5.402790e+01
2,1,[15],5,20,3,1.354337,0.569836,2.376714,143.128411,1.777160e+02,0.805377,2.908944e+02,3.568226e+02,0.815235,0.713795,-61.222413,-4.370028e+01
3,1,[15],5,20,3,1.487676,1.091290,1.363227,99.119417,1.990707e+02,0.497911,3.291539e+02,4.333328e+02,0.759587,1.027637,-42.641951,-4.382043e+01
4,1,[15],5,20,3,5.585796,7.459553,0.748811,440.582588,6.151076e+02,0.716269,6.447628e+03,1.363845e+04,0.472754,3.702599,677.590790,2.508847e+03
5,1,[15],5,20,3,0.941567,0.133829,7.035578,-58.699896,1.861487e+01,-3.153387,-5.470091e+01,1.650757e+01,-3.313687,0.867524,-51.585626,-4.475178e+01
6,1,[15],5,20,3,2898.979615,7664.030328,0.378258,485066.773129,1.252713e+06,0.387213,2.391041e+08,6.207190e+08,0.385205,8775.490862,844.953978,7.414886e+06
7,1,[15],5,20,3,17809.298271,23457.031539,0.759231,666661.788115,1.480543e+06,0.450282,8.406682e+09,1.835576e+10,0.457986,0.685706,-53.830785,-3.691210e+01
8,1,[15],5,20,3,6418.627593,15618.720598,0.410957,19350.444326,4.244095e+04,0.455938,2.855651e+08,8.945427e+08,0.319230,1.243848,34.120195,4.244035e+01


In [23]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [24]:
print(f'len(df.columns): {len(df.columns)}')
print(f'df.columns: {df.columns}')

len(df.columns): 17
df.columns: Index(['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'sym_freq_cnt', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_retnStd/UI', 'SPY_CAGR/retnStd', 'SPY_CAGR/UI'], dtype='object')


In [25]:
# pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
        .agg({'grp(CAGR/retnStd)_mean':     ['mean', 'std'],
              'grp(CAGR/retnStd)_mean/std': ['mean', 'std'],
              'SPY_CAGR/retnStd':           ['mean', 'std']})

In [26]:
tbl

grp(CAGR/retnStd)_mean                grp(CAGR/retnStd)_mean/std           SPY_CAGR/retnStd            
                                                        mean            std                       mean       std             mean         std
days_lookbacks days_eval sym_freq_cnt                                                                                                        
[15]           5         3               130255.061208        256759.895643         0.103813            1.230951       146.488615  353.835656

In [27]:
tbl['SPY_m/s'] = tbl['SPY_CAGR/retnStd', 'mean'] / tbl['SPY_CAGR/retnStd', 'std'] 
tbl['grp-SPY_m/s'] = tbl['grp(CAGR/retnStd)_mean/std', 'mean'] - tbl['SPY_m/s'] 
tbl.sort_values(by='grp-SPY_m/s', ascending=False, inplace=True)
# tbl.sort_values(by='days_lookbacks', ascending=False, inplace=True)
tbl

grp(CAGR/retnStd)_mean                grp(CAGR/retnStd)_mean/std           SPY_CAGR/retnStd               SPY_m/s grp-SPY_m/s
                                                        mean            std                       mean       std             mean         std                      
days_lookbacks days_eval sym_freq_cnt                                                                                                                              
[15]           5         3               130255.061208        256759.895643         0.103813            1.230951       146.488615  353.835656  0.414002   -0.310189

In [28]:
# # https://stackoverflow.com/questions/31674195/plot-normal-distribution-given-mean-and-sigma-python
# # loc is mean, scale is standard deviation
# import pylab
# import numpy as np
# from scipy.stats import norm
# # x = np.linspace(-10000,100000,1000)
# x = np.linspace(-40e+10,50e+10,1000)
# y = norm.pdf(x, loc=2.562777e+10, scale=1.036925e+11)    # loc = mean, scale = standard deviation
# # z = norm.pdf(x, loc=3.540615e+10, scale=1.194430e+11)    # for example
# # z1 = norm.pdf(x, loc=298.805901, scale=826.875749)    # for example
# # z1 = norm.pdf(x, loc=1.021825, scale=1.505096)    # for example
# pylab.plot(x,y, 'b')
# # pylab.plot(x,z, 'g')
# # pylab.plot(x,z1, 'r')
# pylab.show()

In [29]:
# # get values of _cols, where grp(CAGR/retnStd)_mean is max after filtering out inf
# _cols = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
# # _df_no_inf = df.loc[df['grp(CAGR/retnStd)_mean'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# # _idx = _df_no_inf['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# _idx = df['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# grp_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# grp_inf_replacement


In [30]:
# # get values of _cols, where SPY_CAGR/retnStd is max after filtering out inf
# _cols = ['SPY_CAGR/retnStd']
# # _df_no_inf = df.loc[df['SPY_CAGR/retnStd'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# _idx = df['SPY_CAGR/retnStd'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# SPY_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# SPY_inf_replacement

In [31]:
# # # https://stackoverflow.com/questions/50773107/how-to-replace-infinite-value-with-maximum-value-of-a-pandas-column
# # replace inf in column grp(CAGR/UI)_mean
# df['grp(CAGR/UI)_mean'].replace(np.inf, grp_inf_replacement['grp(CAGR/UI)_mean'], inplace=True)
# # replace NaN in column grp(CAGR/UI)_std
# df['grp(CAGR/UI)_std'].replace(np.nan, grp_inf_replacement['grp(CAGR/UI)_std'], inplace=True)
# # replace NaN in column grp(CAGR/UI)_mean/std
# df['grp(CAGR/UI)_mean/std'].replace(np.nan, grp_inf_replacement['grp(CAGR/UI)_mean/std'], inplace=True)
# # replace inf in column SPY_CAGR/UI
# df['SPY_CAGR/UI'].replace(np.inf, SPY_inf_replacement, inplace=True)
# df
# df

In [32]:
# print(f'n_samples:             {n_samples:>13}')
# print(f'days_lookbacks:        {days_lookbacks}')
# print(f'days_eval:             {days_eval:>13}')
# print(f'n_top_syms:            {n_top_syms:>13}')
# print(f'syms_start:            {syms_start:>13}')
# print(f'syms_end:              {syms_end:>13}')
# print(f'grp(CAGR/UI)_mean:     {grp_CAGR_d_UI[0]:>13,.3f}')
# print(f'grp(CAGR/UI)_std:      {grp_CAGR_d_UI[1]:>13,.3f}')
# print(f'grp(CAGR/UI)_mean/std: {grp_CAGR_d_UI[2]:>13,.3f}')
# print(f'SPY_CAGR/UI:           {SPY_CAGR_d_UI[0]:>13,.3f}')